#### Question 2 (Building own NN)

In [ ]:
import numpy as np
import pandas as pd
import random
import math
from sklearn.model_selection import train_test_split
import time
import tensorflow 



x_train = np.load('/kaggle/input/cisc6000-deep-learning-hw1/mnist.train.npy')
submission = pd.read_csv('/kaggle/input/cisc6000-deep-learning-hw1/sample_submission.csv')
y_train = np.load('/kaggle/input/cisc6000-deep-learning-hw1/mnist.trainlabel.npy')
x_test = np.load('/kaggle/input/cisc6000-deep-learning-hw1/mnist.test.npy')

def seed_everything(SEED): 
    np.random.seed(SEED) 
    random.seed(SEED)

seed_everything(2425)

# prepare training (80%) and validation data (20%)
x_train, x_validation, y_train, y_validation = train_test_split(x_train,y_train,test_size = 0.2)

# data reshape
x_train = x_train.reshape(x_train.shape[0], 784)
#y_train = to_categorical(y_train)


shape = (y_train.size, y_train.max()+1)
one_hot = np.zeros(shape)

rows = np.arange(y_train.size)
one_hot[rows, y_train] = 1
y_train = one_hot



shape = (y_validation.size, y_validation.max()+1)
one_hot = np.zeros(shape)

rows = np.arange(y_validation.size)
one_hot[rows, y_validation] = 1
y_validation = one_hot




    
# Activation functions 
def tanh(x,derivative = False):
    if derivative:
        return 1.0 - np.tanh(x)**2  
    return np.tanh(x)

# Softmax function 
def softmax(x, derivative = False):
    exps = np.exp(x - x.max())
    if derivative:
        return exps / np.sum(exps, axis=0) * (1 - exps / np.sum(exps, axis=0))    
    return exps / np.sum(exps, axis=0)




class NN:
    

    def __init__(self, input_neurons, output_neurons_first, output_neurons_second, last_neurons):
        self.input_neurons = input_neurons
        self.output_neurons_first = output_neurons_first
        self.output_neurons_second = output_neurons_second
        self.last_neurons = last_neurons
            
        self.w1 = np.random.rand(self.input_neurons, self.output_neurons_first)
        self.w2 = np.random.rand(self.output_neurons_first, self.output_neurons_second)
        self.w3 = np.random.rand(self.output_neurons_second,self.last_neurons)
                                
            


    def forward_propagation(self,x_train):
        z1 = np.dot(x_train, self.w1)
        a1 = tanh(z1)
        # hiddden 1 > hidden 2
        z2 = np.dot(a1, self.w2)
        a2 = tanh(z2)
        # hidden 2 > output
        z3 = np.dot(a2, self.w3)
        a3 = softmax(z3)
        
        return a3 


    def backward_propagation(self, y_train, output):
        
        updated_weight = {}

        # w3 update
        error = output - y_train 
        updated_weight['w3'] = np.dot(error, a3)

        # w2 update
        error = np.multiply(np.dot(self.w3.T, error), self.tanh(z2, derivative = True))
        updated_weight['w2'] = np.dot(error, a2)

        # w1 update 

        error = np.multiply(np.dot(self.w2.T, error), self.tanh(z1, derivative = True))
        updated_weight['w1'] = np.dot(error, a1)


        return updated_weight 
    
    
    
    
    def update_network_parameters(self, updated_weight):

        for key, value in updated_weight.items():
            for w_arr in updated_weight:
                w_arr -= self.l_rate * value
                
                
    def compute_accuracy(self, x_validaton, y_validaton):
      
        predictions = []

        for x, y in zip(x_validaton, y_validaton):
            output = self.forward_propagation(x)
            pred = np.argmax(output)
            predictions.append(pred == y)
            
            
    def train(self, x_train, y_train, x_validaton, y_validaton):
        start_time = time.time()
        for iteration in range(50):
            for x,y in zip(x_train, y_train):
                output = self.forward_propagation(x)
                updated_weight = self.backward_propagation(y, output)
                self.update_network_parameters(updated_weight)

            accuracy = self.compute_accuracy(x_validaton, x_validaton)


dnn = NN(784, 512, 256, 10)
dnn.train(x_train, y_train, x_validation, y_validation)


#### Question 3

#### Without Augmentation


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import math
import random
import os
import time
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils

x_train = np.load('/kaggle/input/cisc6000-deep-learning-hw1/mnist.train.npy')
submission = pd.read_csv('/kaggle/input/cisc6000-deep-learning-hw1/sample_submission.csv')
y_train = np.load('/kaggle/input/cisc6000-deep-learning-hw1/mnist.trainlabel.npy')
x_test = np.load('/kaggle/input/cisc6000-deep-learning-hw1/mnist.test.npy')

def seed_everything(SEED): 
    np.random.seed(SEED) 
    random.seed(SEED)

seed_everything(2425)

x_train, x_validation, y_train, y_validation = train_test_split(x_train,y_train,test_size = 0.2)

print(x_train.shape)
print(x_validation.shape)
print(y_train.shape)
print(y_validation.shape)



x_train = np.reshape(x_train, [-1, 28, 28, 1])
print(x_train.shape)

x_validation = np.reshape(x_validation, [-1, 28, 28, 1])
print(x_validation.shape)



#x_train = x_train.reshape(44800, 784)
#x_validation = x_validation.reshape(11200, 784)
#x_train= x_train.astype('float32')
#x_validation = x_validation.astype('float32')
#x_test = x_test.reshape(28000, 784)



# normalize data for trainining-purpose 



print('x shape')
print(x_train.shape)
print(x_validation.shape)
print(x_test.shape)
print("-"*100)
print("y_train unique values unpack")
print(np.unique(y_train, return_counts = True))
print("y_validation unique values unpack")
print(np.unique(y_validation, return_counts = True))
print("-"*100)


# One hot encoding
print('y shape')
num_class = 10 
print('before one hot encoding:', y_train.shape)
print('one hot encoding in process......................')
y_train = np_utils.to_categorical(y_train)
y_validation = np_utils.to_categorical(y_validation)

print(y_train.shape)
print('after one hot ecndoing', y_train.shape)


print(x_train.shape)
print(x_validation.shape)
print(y_train.shape)
print(y_validation.shape)

x_train= x_train.astype('float32')
x_validation = x_validation.astype('float32')
x_test = x_test.astype('float32')

#x_train /= 255 
#x_validation /= 255
#x_test /= 255 



model = Sequential()
model.add(Flatten(input_shape = (28,28,1)))
model.add(Dense(512))
model.add(Activation('tanh'))
    
model.add(Dense(256))
model.add(Activation('tanh'))
    
model.add(Dense(10))
model.add(Activation('softmax'))
    
model = model 

    
# Compile model 

opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'], optimizer = opt)


#### saving metrics in history
history = model.fit(x_train, y_train, batch_size = 32, epochs= 50, verbose = 2)

# model save 

save_dir = "./"
model_name = 'h1_result_RMSprop_augmented'
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

(44800, 28, 28)
(11200, 28, 28)
(44800,)
(11200,)
(44800, 28, 28, 1)
(11200, 28, 28, 1)
x shape
(44800, 28, 28, 1)
(11200, 28, 28, 1)
(28000, 28, 28)
----------------------------------------------------------------------------------------------------
y_train unique values unpack
(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([4458, 5096, 4483, 4574, 4350, 4024, 4340, 4724, 4350, 4401]))
y_validation unique values unpack
(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([1128, 1239, 1068, 1129, 1069, 1037, 1126, 1158, 1127, 1119]))
----------------------------------------------------------------------------------------------------
y shape
before one hot encoding: (44800,)
one hot encoding in process......................
(44800, 10)
after one hot ecndoing (44800, 10)
(44800, 28, 28, 1)
(11200, 28, 28, 1)
(44800, 10)
(11200, 10)


2022-10-11 03:22:11.727484: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-11 03:22:11.823608: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-11 03:22:11.824500: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-11 03:22:11.826122: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Epoch 1/50
1400/1400 - 4s - loss: 0.4639 - accuracy: 0.8627
Epoch 2/50
1400/1400 - 3s - loss: 0.4828 - accuracy: 0.8576
Epoch 3/50
1400/1400 - 2s - loss: 0.5083 - accuracy: 0.8558
Epoch 4/50
1400/1400 - 2s - loss: 0.5121 - accuracy: 0.8538
Epoch 5/50
1400/1400 - 2s - loss: 0.5310 - accuracy: 0.8499
Epoch 6/50
1400/1400 - 2s - loss: 0.5487 - accuracy: 0.8491
Epoch 7/50
1400/1400 - 2s - loss: 0.5191 - accuracy: 0.8549
Epoch 8/50
1400/1400 - 2s - loss: 0.5148 - accuracy: 0.8593
Epoch 9/50
1400/1400 - 2s - loss: 0.5245 - accuracy: 0.8565
Epoch 10/50
1400/1400 - 2s - loss: 0.4897 - accuracy: 0.8694
Epoch 11/50
1400/1400 - 2s - loss: 0.4845 - accuracy: 0.8697
Epoch 12/50
1400/1400 - 2s - loss: 0.4734 - accuracy: 0.8740
Epoch 13/50
1400/1400 - 2s - loss: 0.4422 - accuracy: 0.8849
Epoch 14/50
1400/1400 - 2s - loss: 0.4647 - accuracy: 0.8775
Epoch 15/50
1400/1400 - 2s - loss: 0.4720 - accuracy: 0.8761
Epoch 16/50
1400/1400 - 3s - loss: 0.4996 - accuracy: 0.8651
Epoch 17/50
1400/1400 - 2s - loss

2022-10-11 03:24:37.685618: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Saved trained model at ./h1_result_RMSprop_augmented 


#### With Augmentation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import math
import random
import os
import time
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils

x_train = np.load('/kaggle/input/cisc6000-deep-learning-hw1/mnist.train.npy')
submission = pd.read_csv('/kaggle/input/cisc6000-deep-learning-hw1/sample_submission.csv')
y_train = np.load('/kaggle/input/cisc6000-deep-learning-hw1/mnist.trainlabel.npy')
x_test = np.load('/kaggle/input/cisc6000-deep-learning-hw1/mnist.test.npy')

def seed_everything(SEED): 
    np.random.seed(SEED) 
    random.seed(SEED)

seed_everything(2425)

x_train, x_validation, y_train, y_validation = train_test_split(x_train,y_train,test_size = 0.2)

print(x_train.shape)
print(x_validation.shape)
print(y_train.shape)
print(y_validation.shape)



x_train = np.reshape(x_train, [-1, 28, 28, 1])
print(x_train.shape)

x_validation = np.reshape(x_validation, [-1, 28, 28, 1])
print(x_validation.shape)



#x_train = x_train.reshape(44800, 784)
#x_validation = x_validation.reshape(11200, 784)
#x_train= x_train.astype('float32')
#x_validation = x_validation.astype('float32')
#x_test = x_test.reshape(28000, 784)



# normalize data for trainining-purpose 



print('x shape')
print(x_train.shape)
print(x_validation.shape)
print(x_test.shape)
print("-"*100)
print("y_train unique values unpack")
print(np.unique(y_train, return_counts = True))
print("y_validation unique values unpack")
print(np.unique(y_validation, return_counts = True))
print("-"*100)


# One hot encoding
print('y shape')
num_class = 10 
print('before one hot encoding:', y_train.shape)
print('one hot encoding in process......................')
y_train = np_utils.to_categorical(y_train)
y_validation = np_utils.to_categorical(y_validation)

print(y_train.shape)
print('after one hot ecndoing', y_train.shape)


print(x_train.shape)
print(x_validation.shape)
print(y_train.shape)
print(y_validation.shape)

x_train= x_train.astype('float32')
x_validation = x_validation.astype('float32')
x_test = x_test.astype('float32')

#x_train /= 255 
#x_validation /= 255
#x_test /= 255 



model = Sequential()
model.add(Flatten(input_shape = (28,28,1)))
model.add(Dense(512))
model.add(Activation('tanh'))
    
model.add(Dense(256))
model.add(Activation('tanh'))
    
model.add(Dense(10))
model.add(Activation('softmax'))
    
model = model 

opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'], optimizer = opt)



from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator()


datagen = ImageDataGenerator(        
        rotation_range = 45,
        horizontal_flip = True,vertical_flip = True 
        )
datagen.fit(x_train)



train_generator = datagen.flow(x_train, y_train, batch_size=32, shuffle=True)
validation_generator = datagen.flow(x_validation, y_validation, batch_size=32, shuffle=True)




start = time.time()


history_inception_datagen = model.fit_generator(train_generator,
                                                steps_per_epoch = 1400,
                                                epochs=50,
                                                validation_data = validation_generator
                                                )



elapsed_time_fl = (time.time() - start)    
print(elapsed_time_fl)


save_dir = "./"
model_name = 'h1_result_RMSprop_augmented'
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)


#### Overall validation accuracy & Loss 

In [ ]:
y_validation = np_utils.to_categorical(y_validation)

mnist_model = load_model(model_name)
loss_and_metrics = mnist_model.evaluate(x_validation, y_validation, verbose = 2)

print("val loss", loss_and_metrics[0])
print("val accuracy", loss_and_metrics[1])

#### Pred_test

In [ ]:
# Each class' accuracy


mnist_model = load_model(model_name)
y_pred = np.argmax(mnist_model.predict(x_validation), axis = 1)

num_correct_class = [0 for i in range(10)]
num_sample_class = [0 for i in range(10)]


for i in range(len(y_pred)):
    label = y_validation[i]
    if y_pred[i] == label:
        num_correct_class[label] +=1
        
    num_sample_class[label]+=1 

    
for i in range(10):
    acc = 100 * num_correct_class[i]/ num_sample_class[i]
    print(f' accuray of class {i} = {acc:.2f}%')

In [ ]:
# test data accuracy

pred_test = np.argmax(mnist_model.predict(x_test), axis = 1)
submission = pd.read_csv('/kaggle/input/cisc6000-deep-learning-hw1/sample_submission.csv',index_col = 0)
submission['class'] = pred_test
submission.to_csv('RMSprop_augmented-Neural_Network_Submission.csv')
submission.head()




In [ ]:
y_pred.shape

In [ ]:
y_validation.shape

In [ ]:
y_validation

In [ ]:
y_pred

In [ ]:
x_validation.shape

In [ ]:
y_train.shape

In [ ]:
y_validation.shape

In [ ]:
shape = (y_train.size, y_train.max()+1)
one_hot = np.zeros(shape)

rows = np.arange(y_train.size)
one_hot[rows, y_train] = 1
y_train = one_hot



shape = (y_validation.size, y_validation.max()+1)
one_hot = np.zeros(shape)

rows = np.arange(y_validation.size)
one_hot[rows, y_validation] = 1
y_validation = one_hot